# Best when try: cat and dog classifier using tensorflow


- 98%
- Source: https://github.com/wtfleming/jupyter-notebooks-public/blob/master/dogs-vs-cats/dogs-vs-cats-part-2.ipynb
- Original run and try in the middle of this notebook: https://colab.research.google.com/drive/1HsvpXG5ks845Mf1hPDBf4pM_aBaLDvph#scrollTo=K92VI7vIlCFk

This notebook is just a copy.

In [ ]:
# In case: Not found: No algorithm worked!
# In case: This is probably because cuDNN failed to initialize

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [ ]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import numpy as np

In [ ]:
path_saved = "../saved_model/"

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing import image

In [ ]:
img_dim = (224,224)

In [ ]:
base_dir = '../dataset/cats_and_dogs_filtered'

train_dir = os.path.join( base_dir, 'train')
validation_dir = os.path.join( base_dir, 'validation')

train_cats_dir = os.path.join(train_dir, 'cats') # Directory with our training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs') # Directory with our training dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats') # Directory with our validation cat pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')# Directory with our validation dog pictures

In [ ]:
test_datagen = ImageDataGenerator( rescale = 1.0/255. )
validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 20,
                                                          class_mode  = 'binary', 
                                                          target_size = img_dim)

In [ ]:
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(224, 224, 3))

conv_base.trainable = True

set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

In [ ]:
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224), # resize all images to 150 x 150
    batch_size=50,
    class_mode='binary') # because we use binary_crossentropy loss we need binary labels

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(224, 224), # resize all images to 150 x 150
    batch_size=50,
    class_mode='binary')

In [ ]:
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-5),
              metrics=['acc'])

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=40,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=20) 

In [ ]:
model.save('cat_dog_best_md_cp1.h5')

In [ ]:
labels = (validation_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
labels

In [ ]:
steps = validation_generator.n // validation_generator.batch_size
steps

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

In [ ]:
# loss
epochs = range(len(loss))
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend(loc=0)
plt.figure()
plt.show()

In [ ]:
# single image
# path = base_dir + "/test/dog4.jpeg"
path = base_dir + "/test/cat.jpeg"
img = image.load_img(path, target_size=img_dim)
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
images = np.vstack([x])

print(model_3.predict(images))

classes = (model_3.predict(images) > 0.5).astype("int32")[0][0]
classes

In [ ]:
# a folder
file_path = '../dataset/cats_and_dogs_filtered/test/' # current dir
test_files = os.listdir(file_path)

for file in test_files:
    # predicting images
    path=file_path + file
    img=image.load_img(path, target_size=img_dim)

    x=image.img_to_array(img)
    x=np.expand_dims(x, axis=0)
    images = np.vstack([x])

    classes = (model_3.predict(images) > 0.5).astype("int32")[0][0]
    print(classes)

    if classes>0:
        print(file + " is a dog")
    else:
        print(file + " is a cat")